Your first option for automating SQL Notebooks is the Invoke-SqlNotebook cmdlet.

In [ ]:
Invoke-SqlNotebook

# I heard you like to run Notebooks

## So I created a PowerShell Notebook to execute a T-SQL Notebook, using the new Invoke-SqlNotebook PowerShell cmdlet

This is a simple demonstration of how your can use a PowerShell Notebook to execute a T-SQL Notebook across all your Registered SQL Servers, using the new Invoke-SqlNotebook cmdlet. The results of the execution against each SQL Server Instance will be saved locally to it's own Output-Notebook, with a unique name.

In [4]:
Import-Module SqlServer;

$SQLDockCred = Get-Credential sa

dir 'SQLSERVER:\SQLRegistration\Database Engine Server Group' |
WHERE { $_.Mode -ne 'd'} |            
foreach {
    Get-SqlInstance -ServerInstance $_.Name -Credential $SQLDockCred |
    foreach {
            Invoke-SqlNotebook -ServerInstance $_.Name -Database master -Credential $SQLDockCred -InputFile "$home\Documents\SQL Server Management Studio\BPCheck.ipynb" `
            -OutputFile "BPCheck_output_$($_.NetName)_$(Get-Date -Date $Date -Format yyyyMMdd).ipynb"
            }
        }


In [8]:
$SQLDockCred = Get-Credential sa

dir 'SQLSERVER:\SQLRegistration\Database Engine Server Group' |
WHERE { $_.Mode -ne 'd'} |            
foreach {
    Get-SqlInstance -ServerInstance $_.Name -Credential $SQLDockCred 
        }



Name                 Status           Size     Space  Recovery Compat. Owner                     Collation             
                                            Available  Model     Level                                                 
----                 ------           ---- ---------- -------- ------- -----                     ---------             
AdventureWorks2016   Normal      209.63 MB    1.13 MB Simple       130 sa                        SQL_Latin1_General_CP1
                                                                                                 _CI_AS                
AdventureWorksDW2012 Normal      167.00 MB   32.47 MB Simple       110 sa                        SQL_Latin1_General_CP1
                                                                                                 _CI_AS                
AdventureWorksDW2014 Normal      188.00 MB   78.10 MB Simple       120 sa                        SQL_Latin1_General_CP1
                                       

In [7]:
$SQLDockCred = Get-Credential sa

dir 'SQLSERVER:QLRegistrationatabase Engine Server Group' |
WHERE { $_.Mode -ne 'd'} |            
foreach {
    Get-SqlInstance -ServerInstance $_.Name -Credential $SQLDockCred |
    foreach {
            Get-SqlDatabase -ServerInstance $_.Name -Credential $SQLDockCred
            }
        }


& { Import-Module SqlServer;
>> 
>> $SQLDockCred = Get-Credential sa
>> 
>> dir 'SQLSERVER:\SQLRegistration\Database Engine Server Group' |
>> WHERE { $_.Mode -ne 'd'} |            
>> foreach {
>>     Get-SqlInstance -ServerInstance $_.Name -Credential $SQLDockCred |
>>     foreach {
>>             Get-SqlDatabase -ServerInstance $_.Name -Credential $SQLDockCred
>>             }
>>         }
>>  }
>> 

Name                 Status           Size     Space  Recovery Compat. Owner                     Collation             
                                            Available  Model     Level                                                 
----                 ------           ---- ---------- -------- ------- -----                     ---------             
AdventureWorks2016   Normal      209.63 MB    1.13 MB Simple       130 sa                        SQL_Latin1_General_CP1
                                                                                                 _CI_AS          